In [69]:
import pandas as pd
import numpy as np
import pybaseball

### Read in BRef data & Calculate Height and Weight Percentiles

In [70]:
pitchers = pd.read_csv("pitchers_br.csv")

def height_to_inches(height_str):
    feet, inches = height_str.split("'")
    inches = inches.replace('"', '').strip()
    return int(feet) * 12 + int(inches)

pitchers['height_inches'] = pitchers['Ht'].apply(height_to_inches)
# pitchers["Wt"].apply(int)

pitchers['height_percentile'] = pitchers['Ht'].rank(pct=True).round(2)
pitchers['weight_percentile'] = pitchers['Wt'].rank(pct=True).round(2)

pitchers["height_zscore"] = (pitchers["height_inches"] - pitchers["height_inches"].mean()) / pitchers["height_inches"].std()
pitchers["weight_zscore"] = (pitchers["Wt"] - pitchers["Wt"].mean()) / pitchers["Wt"].std()


pitchers_sorted = pitchers.sort_values(by='height_zscore', ascending=False)
print(pitchers_sorted)


     Unnamed: 0                Name      Ht   Wt  height_inches  \
360         360        Paul Gervase  6' 10"  230             82   
416         416         Bailey Ober   6' 9"  260             81   
340         340       Tyler Glasnow   6' 8"  225             80   
372         372          Eury Pérez   6' 8"  220             80   
648         648           Joe Boyle   6' 8"  250             80   
..          ...                 ...     ...  ...            ...   
343         343  Yoshinobu Yamamoto  5' 10"  176             70   
677         677      Danny Coulombe  5' 10"  190             70   
171         171            Sam Moll   5' 9"  190             69   
63           63         Rico Garcia   5' 9"  201             69   
555         555         Yuki Matsui   5' 8"  165             68   

     height_percentile  weight_percentile  height_zscore  weight_zscore  
360               0.26               0.81       3.355894       0.819287  
416               1.00               0.98      

### Calculate Size Score

In [71]:
pitchers["size_score"] = 10 * pitchers["height_percentile"] + 90 * pitchers["weight_percentile"]
# pitchers_sorted = pitchers.sort_values(by='size_score', ascending=False)
# print(pitchers_sorted)

In [72]:
# Alternative Calculation of Size score: z-scores
pitchers["size_score"] = 10 * pitchers["height_zscore"] + 90 * pitchers["weight_zscore"]
# pitchers_sorted = pitchers.sort_values(by='size_score', ascending=False)
# print(pitchers_sorted)

### Big-Body Score as an alternative to Size Score

Capture a score somewhere between BMI and Corpulence Index (using Imperial units - percentiles will be calculated anyway)
Big-Body Score = $\frac{Weight (lbs)}{(Height (in)^{2.5})}$

In [92]:
pitchers["big_body_raw"] = pitchers["Wt"] / (pitchers["height_inches"])**2.5
# pitchers['big_body_score'] = pitchers['big_body_raw'].rank(pct=True).round(4) * 100
pitchers["big_body_score"] = (pitchers["big_body_raw"] - pitchers["big_body_raw"].mean()) / pitchers["big_body_raw"].std() # + 0.5 * pitchers["height_zscore"]
pitchers_sorted = pitchers.sort_values(by='big_body_score', ascending=False)
print(pitchers_sorted[["Name", "big_body_score"]])

                 Name  big_body_score
164    Tony Santillan        3.517482
348  Brusdar Graterol        3.445045
43        Joe Jiménez        3.124514
146     Dan Altavilla        2.756605
467      David Bednar        2.656721
..                ...             ...
527      Jean Cabrera       -2.594183
3        Juan Morillo       -2.598771
483     Michael Arias       -2.622646
252    Keider Montero       -2.861545
453  Huascar Brazobán       -2.868251

[742 rows x 2 columns]


### Calculate Dominance Score from stats
This year's stats from FanGraphs

In [74]:
pitching_stats = pd.read_csv("pitchers_fg.csv")

In [75]:
# set a 10-game qualification filter
pitching_stats = pitching_stats[pitching_stats["G"] > 10]

# determine starters/relievers
pitching_stats["Pos"] = np.where(
    pitching_stats["GS"] > 0.5 * pitching_stats["G"], 
    "SP", 
    "RP"
)

In [76]:
pitching_stats.columns



Index(['Unnamed: 0', 'Name', 'WAR', 'G', 'GS', 'SO', 'K/9', 'FIP', 'K%',
       'FB% 2', 'vFA (sc)', 'vSI (sc)', 'vFC (sc)', 'Zone%', 'SwStr%',
       'Stuff+', 'Pitching+', 'Pos'],
      dtype='object')

In [94]:
# pitching_stats_sp = pd.DataFrame(pitching_stats[pitching_stats["Pos"] == "SP"])
# pitching_stats_sp['SO_percentile'] = pitching_stats_sp['SO'].rank(pct=True).round(2)
# pitching_stats_sp['SO9_percentile'] = pitching_stats_sp['K/9'].rank(pct=True).round(2)
# pitching_stats_rp = pd.DataFrame(pitching_stats[pitching_stats["Pos"] == "RP"])
# pitching_stats_rp['SO_percentile'] = pitching_stats_rp['SO'].rank(pct=True).round(2)
# pitching_stats_rp['SO9_percentile'] = pitching_stats_rp['K/9'].rank(pct=True).round(2)

import pandas as pd

# SP pitchers
pitching_stats_sp = pd.DataFrame(pitching_stats[pitching_stats["Pos"] == "SP"])
pitching_stats_sp['SO_z'] = (pitching_stats_sp['SO'] - pitching_stats_sp['SO'].mean()) / pitching_stats_sp['SO'].std()
pitching_stats_sp['SO9_z'] = (pitching_stats_sp['K/9'] - pitching_stats_sp['K/9'].mean()) / pitching_stats_sp['K/9'].std()

# RP pitchers
pitching_stats_rp = pd.DataFrame(pitching_stats[pitching_stats["Pos"] == "RP"])
pitching_stats_rp['SO_z'] = (pitching_stats_rp['SO'] - pitching_stats_rp['SO'].mean()) / pitching_stats_rp['SO'].std()
pitching_stats_rp['SO9_z'] = (pitching_stats_rp['K/9'] - pitching_stats_rp['K/9'].mean()) / pitching_stats_rp['K/9'].std()


pitching_stats = pd.concat([pitching_stats_sp, pitching_stats_rp], ignore_index=True)

pitching_stats["fb_velo"] = pitching_stats[["vFA (sc)", "vSI (sc)", "vFC (sc)",]].max(axis=1, skipna=True)
pitching_stats['fbv_z'] = (pitching_stats['fb_velo'] - pitching_stats['fb_velo'].mean()) / pitching_stats['fb_velo'].std()

pitching_stats["Stuff+_z"] = (pitching_stats["Stuff+"] - pitching_stats["Stuff+"].mean()) / pitching_stats["Stuff+"].std()
pitching_stats["Pitching+_z"] = (pitching_stats["Pitching+"] - pitching_stats["Pitching+"].mean()) / pitching_stats["Pitching+"].std()



# pitching_stats["dominance_score_raw"] = 10 * pitching_stats["SO_percentile"] + 80 * pitching_stats["SO9_percentile"] + 50 * pitching_stats["fbv_percentile"] + pitching_stats["Stuff+"]
# pitching_stats["dominance_score"] = pitching_stats['dominance_score_raw'].rank(pct=True).round(4) * 100
# pitching_stats["dominance_score_raw"] = 10 * pitching_stats["SO9_z"] + 20 * pitching_stats["fbv_z"] + 30 * (pitching_stats["Stuff+"] / 100)
pitching_stats["dominance_score_raw"] = pitching_stats["Pitching+_z"] + pitching_stats["fbv_z"]
pitching_stats["dominance_score"] = (pitching_stats['dominance_score_raw'] - pitching_stats['dominance_score_raw'].mean()) / pitching_stats['dominance_score_raw'].std()


print(pitching_stats.sort_values(by='dominance_score', ascending=False).head(5))

     Unnamed: 0             Name  WAR   G  GS  SO    K/9   FIP     K%  FB% 2  \
162           0  Aroldis Chapman  1.8  51   0  66  12.73  1.95  0.379  0.749   
161         127    Robert Suarez  1.9  53   0  53   9.29  2.49  0.262  0.761   
183          27    Trevor Megill  1.2  45   0  51  10.93  2.68  0.304  0.609   
159          18   Adrian Morejon  2.0  58   0  55   9.00  1.95  0.258  0.646   
185         180     Mason Miller  1.2  44   0  73  14.71  2.71  0.420  0.546   

     ...  Pitching+  Pos      SO_z     SO9_z  fb_velo     fbv_z  \
162  ...        156   RP  1.351607  1.887128     99.7  2.172039   
161  ...        159   RP  0.408779  0.163502     98.5  1.668166   
183  ...        148   RP  0.263728  0.985231     99.1  1.920103   
159  ...        141   RP  0.553829  0.018196     97.6  1.290262   
185  ...        118   RP  1.859284  2.879216    101.2  2.801880   

     dominance_score_raw dominance_score  Stuff+_z  Pitching+_z  
162             5.865015        3.470290  1.796183

### Calculate HOSS = Size Score + Dominance Score

In [95]:
pitchers_hoss = pd.merge(
    pitching_stats, 
    pitchers, 
    on="Name",
    how="left"
)

# pitchers["HOSS"] = pitchers["size_score"] + pitchers["dominance_score"]
pitchers_hoss["HOSS"] = pitchers_hoss["big_body_score"] + pitchers_hoss["dominance_score"]

pitchers_sorted = pitchers_hoss.sort_values(by='HOSS', ascending=False)
print(pitchers_sorted[["Name", "big_body_score", "dominance_score", "HOSS"]].head(10))

                Name  big_body_score  dominance_score      HOSS
163  Aroldis Chapman        0.685451         3.470290  4.155741
189  Graham Ashcraft        2.115739         1.939895  4.055634
165     David Bednar        2.656721         1.023291  3.680013
244   Tony Santillan        3.517482         0.153524  3.671006
262      Luis Garcia        2.551611         1.030622  3.582233
162    Robert Suarez        0.185439         3.294949  3.480388
22    Drew Rasmussen        2.131172         1.330384  3.461556
256     Ryan Helsley        1.455373         1.881251  3.336624
284     Tanner Scott        2.301708         0.906398  3.208106
0       Tarik Skubal        1.307036         1.859457  3.166493
